In [1]:
import graphlab 
from graphlab import SFrame

In [11]:
data=SFrame(data='/Volumes/SILVA/新媒体数据/待统计处理0407/合并表3.csv')
train_data,test_data=data.random_split(0.85,seed=1)
data

Finished parsing file /Volumes/SILVA/新媒体数据/待统计处理0407/合并表3.csv

Parsing completed. Parsed 100 lines in 0.010209 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,int,int,int,int,int,float]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Volumes/SILVA/新媒体数据/待统计处理0407/合并表3.csv

Parsing completed. Parsed 1077 lines in 0.010287 secs.

id,Words_of_the_Title,English_or_not,the_Length_of_one_Episode,the_Playback_Amount_of_one_Episode ...,Release_Week
344,15,0,194,18,3
737,15,0,253,22,3
741,10,0,374,30,3
738,13,0,247,44,3
731,15,0,253,69,4
735,19,0,268,53,3
766,10,0,231,53,3
730,13,0,247,73,4
748,14,0,211,65,3
424,8,0,51,22,1


In [13]:
graphlab.canvas.set_target('ipynb')

In [15]:
my_features = ['Words_of_the_Title', 'the_Length_of_one_Episode','English_or_not']

In [16]:
data[my_features].show()

In [18]:
data.show(view="Scatter Plot", x="Words_of_the_Title", y="perweek")
data.show(view="Scatter Plot", x="the_Length_of_one_Episode", y="perweek")

In [21]:
data['Words_of_the_Title'].show()
data['the_Length_of_one_Episode'].show()

In [5]:
my_features = ['Words_of_the_Title', 'the_Length_of_one_Episode','English_or_not']

In [28]:
model = graphlab.linear_regression.create(train_data, target = 'perweek', features =my_features, 
                                                  validation_set = None,verbose=True)
model.summary()

Linear regression:

--------------------------------------------------------

Number of examples          : 914

Number of features          : 3

Number of unpacked features : 3

...

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.001362     | 2318456.565718     | 163259.855279 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

Class                          : LinearRegression

Schema
------
Number of coefficients         : 4
Number of examples             : 914
Number of feature columns      : 3
Number of unpacked features    : 3

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.01

Training Summary
----------------
Solver                         : newton
Solver iterations              : 1
Solver status                  : SUCCESS: Optimal solution found.
Training time (sec)            : 0.0024

Settings
--------
Residual sum of squares        : 2.43615552361e+13
Training RMSE                  : 163259.8553

Highest Positive Coefficients
-----------------------------
English_or_not                 : 6570.5599
Words_of_the_Title             : 5219.5702

Lowest Negative Coefficients
----------------------------
(intercept)                    : -3868.5246
the_Length_of_one_Episode      : -20.3413



In [29]:
weight_summary = model.get("coefficients")
print weight_summary

+---------------------------+-------+----------------+---------------+
|            name           | index |     value      |     stderr    |
+---------------------------+-------+----------------+---------------+
|        (intercept)        |  None | -3868.52458472 | 17242.4855139 |
|     Words_of_the_Title    |  None | 5219.57024704  | 991.572491962 |
| the_Length_of_one_Episode |  None | -20.3413496889 | 35.3505813083 |
|       English_or_not      |  None | 6570.55992764  | 16889.3808068 |
+---------------------------+-------+----------------+---------------+
[4 rows x 4 columns]



In [30]:
predictions = model.predict(train_data)

In [31]:
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions
    predictions = model.predict(data)
    # Then compute the residuals/errors
    residuals= outcome-predictions
    # Then square and add them up
    RSS = sum(residuals**2)
    return(RSS)    

In [85]:
rss_test = get_residual_sum_of_squares(model, train_data, train_data['perweek'])
print rss_test

2.43615552361e+13


# Poly data

In [34]:
def polynomial_sframe_1(feature, degree):
    poly_sframe = graphlab.SFrame()
    poly_sframe['Words_of_the_Title']= feature
    if degree > 1:
        for power in range(2, degree+1): 
            name = 'Words_of_the_Title' + str(power)
            poly_sframe[name]=feature.apply(lambda x: x**power)
    return poly_sframe

In [35]:
def polynomial_sframe_2(feature, degree):
    poly_sframe = graphlab.SFrame()
    poly_sframe['the_Length_of_one_Episode']= feature
    if degree > 1:
        for power in range(2, degree+1): 
            name = 'the_Length_of_one_Episode' + str(power)
            poly_sframe[name]=feature.apply(lambda x: x**power)
    return poly_sframe

In [96]:
polydata1=polynomial_sframe_1(train_data['Words_of_the_Title'],3)
my_features1 = polydata1.column_names()
polydata1

Words_of_the_Title,Words_of_the_Title2,Words_of_the_Title3
15,225,3375
15,225,3375
10,100,1000
13,169,2197
15,225,3375
19,361,6859
10,100,1000
13,169,2197
16,256,4096
30,900,27000


In [97]:
polydata2=polynomial_sframe_2(train_data['the_Length_of_one_Episode'],3)
my_features2 = polydata2.column_names()
polydata2
my_features2

['the_Length_of_one_Episode',
 'the_Length_of_one_Episode2',
 'the_Length_of_one_Episode3']

In [98]:
polydata=polydata1
polydata

Words_of_the_Title,Words_of_the_Title2,Words_of_the_Title3
15,225,3375
15,225,3375
10,100,1000
13,169,2197
15,225,3375
19,361,6859
10,100,1000
13,169,2197
16,256,4096
30,900,27000


In [99]:
polydata[my_features2]=polydata2[my_features2]
polydata['English_or_not']=train_data['English_or_not']
my_features3 = polydata.column_names()
my_features3

['Words_of_the_Title',
 'Words_of_the_Title2',
 'Words_of_the_Title3',
 'the_Length_of_one_Episode',
 'the_Length_of_one_Episode2',
 'the_Length_of_one_Episode3',
 'English_or_not']

In [100]:
polydata1['perweek'] = train_data['perweek']

In [111]:
polymodel = graphlab.linear_regression.create(polydata, target = 'perweek', features =my_features3, 
                                                  validation_set = None,verbose=True,l2_penalty=1e1)
polymodel.summary()

Linear regression:

--------------------------------------------------------

Number of examples          : 914

Number of features          : 7

Number of unpacked features : 7

...

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.002753     | 2313276.411128     | 162356.794038 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

Class                          : LinearRegression

Schema
------
Number of coefficients         : 8
Number of examples             : 914
Number of feature columns      : 7
Number of unpacked features    : 7

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 10.0

Training Summary
----------------
Solver                         : newton
Solver iterations              : 1
Solver status                  : SUCCESS: Optimal solution found.
Training time (sec)            : 0.0045

Settings
--------
Residual sum of squares        : 2.40927919132e+13
Training RMSE                  : 162356.794

Highest Positive Coefficients
-----------------------------
English_or_not                 : 12925.763
Words_of_the_Title             : 2241.3878
the_Length_of_one_Episode      : 166.1603
Words_of_the_Title2            : 156.773

Lowest Negative Coefficients
----------------------------
(intercept)                    : -8369.1699
Words_of_the_Title3   

In [112]:
weight_summary = polymodel.get("coefficients")
print weight_summary

+----------------------------+-------+--------------------+-------------------+
|            name            | index |       value        |       stderr      |
+----------------------------+-------+--------------------+-------------------+
|        (intercept)         |  None |   -8369.16994609   |   59918.9133156   |
|     Words_of_the_Title     |  None |   2241.38782909    |   10327.7325217   |
|    Words_of_the_Title2     |  None |   156.773007266    |   513.976592432   |
|    Words_of_the_Title3     |  None |   -3.02883204748   |   7.79302070168   |
| the_Length_of_one_Episode  |  None |   166.160340075    |   224.494469401   |
| the_Length_of_one_Episode2 |  None |  -0.221025072221   |   0.633467320239  |
| the_Length_of_one_Episode3 |  None | -8.53028530552e-05 | 0.000491661923028 |
|       English_or_not       |  None |   12925.7630052    |   17168.2348477   |
+----------------------------+-------+--------------------+-------------------+
[8 rows x 4 columns]



In [116]:
for l2_penalty in [1e-25, 1e-10, 1e-6, 1e-3, 1e1]:
    polymodel = graphlab.linear_regression.create(polydata, target = 'perweek', features =my_features3, 
                                                  validation_set = None,verbose=False,l2_penalty=l2_penalty)
    rss_test = get_residual_sum_of_squares(polymodel, train_data, train_data['perweek'])
    print rss_test

4.69453886713e+13
4.69453886639e+13
4.69453146831e+13
4.68716318589e+13
2.5287526302e+13
